In [1]:
import os
print(f"Number of CPUs using os: {os.cpu_count()}")

Number of CPUs using os: 8


In [ ]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO, A2C, SAC, TD3
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat_simple import ForexTradingEnv
# from trading.environments.forex_env_flat_multi_pair import MultipairForexTradingEnv

from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager
from sb3_contrib import RecurrentPPO

source_path = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/df_with_all_indics_unbiased/not_norm/train2/'
source_dfs = [os.path.join(source_path, f) for f in os.listdir(source_path) if f.endswith('.parquet') and not f.startswith('.') and 'validate' not in f]

eval_path = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/df_with_all_indics_unbiased/not_norm/train2/EUR_GBP_validate.parquet'
sequence = 5
saving_path = f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/df_with_all_indics_unbiased/not_norm/train/results/'
os.makedirs(saving_path, exist_ok=True)



def make_train_env():
    env = ForexTradingEnv(
        df_paths=source_dfs,
        eval_mode=False,
        sequence_length=sequence,


    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True)
    return env

def make_eval_env():
    env = ForexTradingEnv(
        df_paths=source_dfs,
        eval_path=eval_path,
        eval_mode=True,
        pair='EUR_GBP',
        sequence_length=sequence,


    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=False)
    env.training = False
    return env

train_env = make_train_env()
eval_env = make_eval_env()

eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=saving_path,
    log_path=saving_path,
    eval_freq=500_000,  # Adjust as needed
    n_eval_episodes=5,
    deterministic=True,
    render=False
)

model = SAC(
    'MlpPolicy',
    train_env,
    verbose=0,
    tensorboard_log=f'{saving_path}sequence_{sequence}_cont_space_SAC_13m_core_500k/',
)
# Define policy kwargs for the LSTM configuration
# policy_kwargs = dict(
#     # Network Architecture
#     net_arch=dict(
#         # Actor (policy) network
#         pi=[256, 128],  # Larger first layer to process high-dimensional input
#         # Critic (value) network
#         vf=[256, 128]   # Match actor architecture for balanced learning
#     ),
    
#     # LSTM Configuration
#     lstm_hidden_size=256,      # Larger hidden size to capture complex patterns
#     n_lstm_layers=2,           # Multiple layers for hierarchical feature learning
#     enable_critic_lstm=True,   # Share temporal understanding between actor and critic
    
#     # LSTM specific parameters
#     lstm_kwargs=dict(
#         dropout=0.2            # Slightly higher dropout for regularization
#     )
# )

# policy_kwargs_complex = dict(
#     net_arch=dict(
#         pi=[512, 256, 128],
#         vf=[512, 256, 128]
#     ),
#     lstm_hidden_size=512,
#     n_lstm_layers=3,
#     enable_critic_lstm=True,
#     lstm_kwargs=dict(
#         dropout=0.25
#     )
# )

# model = RecurrentPPO(
#     'MlpLstmPolicy',
#     train_env,
#     verbose=0,
#     tensorboard_log=f'{saving_path}sequence_{sequence}_RecurrentPPO/',
#     policy_kwargs=policy_kwargs,
# )
model.learn(
    total_timesteps=13_000_000,  # Adjust as needed
    callback=eval_callback
)

model.save(f'{saving_path}{sequence}_best_model_core.zip')
train_env.save(f'{saving_path}{sequence}_vec_normalize_core.pkl')


Logging configuration loaded from /Users/floriankockler/Code/GitHub.nosync/ai6-gcp-bot/forex_trading_system/config/logging_config.yaml
2024-12-12 15:31:43 - ForexEnv2_flat - INFO - forex_env2_flat_simple.py:507 - Selected features (15): ['price_change', 'close_norm', 'rsi', 'stoch_d', 'bb_position', 'bb_width_normalized', 'dist_sma_20_atr_adj', 'dist_sma_50_atr_adj', 'macd_hist_norm', 'di_spread', 'trend_strength', 'atr_regime_20', 'momentum_1h', 'momentum_4h', 'close']
2024-12-12 15:31:44 - ForexEnv2_flat - INFO - forex_env2_flat_simple.py:507 - Selected features (15): ['price_change', 'close_norm', 'rsi', 'stoch_d', 'bb_position', 'bb_width_normalized', 'dist_sma_20_atr_adj', 'dist_sma_50_atr_adj', 'macd_hist_norm', 'di_spread', 'trend_strength', 'atr_regime_20', 'momentum_1h', 'momentum_4h', 'close']
2024-12-12 15:31:44 - ForexEnv2_flat - INFO - forex_env2_flat_simple.py:507 - Selected features (15): ['price_change', 'close_norm', 'rsi', 'stoch_d', 'bb_position', 'bb_width_normalize

In [1]:

import numpy as np
import os, sys
# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat_simple import ForexTradingEnv

source_path = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/df_with_all_indics_unbiased/not_norm/train/'
source_dfs = [os.path.join(source_path, f) for f in os.listdir(source_path) if f.endswith('.parquet') and not f.startswith('.') and 'validate' not in f]





# Create the environment
env = ForexTradingEnv(
    df_paths=source_dfs,
        eval_mode=False,
        sequence_length=5,
        initial_balance=1_000_000.0
)  # Add any necessary parameters here



# Initialize statistics
total_rewards = []
episode_lengths = []

# Run 100 episodes
step_count = 0
observation = env.reset()
for episode in range(5):
    episode_reward = 0

    action = env.action_space.sample()
    print(f"\nStep {step_count + 1}")
    print(f"Taking action: {action}")
    
    # Take step in environment
    try:
        next_observation, reward, terminated, truncated, info = env.step(action)
        
        # Print post-step information
        print(f"Reward: {reward}")
        print(f"truncated: {truncated}")
        print(f"Info: {info}")

        episode_reward += reward
        step_count += 1
             
    except Exception as e:
        print(f"Error occurred during step: {e}")
        break





Logging configuration loaded from /Users/floriankockler/Code/GitHub.nosync/ai6-gcp-bot/forex_trading_system/config/logging_config.yaml
2024-12-12 10:14:03 - ForexEnv2_flat - INFO - forex_env2_flat_simple.py:503 - Selected features (37): ['close', 'rsi', 'roc_10', 'stoch_rsi', 'stoch_k', 'stoch_d', 'bb_upper', 'bb_middle', 'bb_lower', 'bb_bandwidth', 'bb_percent', 'atr', 'plus_di', 'minus_di', 'adx', 'price_change', 'close_norm', 'hl_range_ratio', 'dist_sma_20_atr_adj', 'dist_sma_50_atr_adj', 'sma_cross_atr', 'bb_position', 'bb_width_normalized', 'tenkan_dist_atr_adj', 'kijun_dist_atr_adj', 'rsi_change', 'macd_hist_norm', 'di_spread', 'trend_strength', 'atr_normalized', 'atr_regime_10', 'atr_regime_20', 'trend_strength_20', 'trend_strength_50', 'momentum_1h', 'momentum_4h', 'price_norm']
2024-12-12 10:14:04 - ForexEnv2_flat - INFO - forex_env2_flat_simple.py:503 - Selected features (37): ['close', 'rsi', 'roc_10', 'stoch_rsi', 'stoch_k', 'stoch_d', 'bb_upper', 'bb_middle', 'bb_lower', '

In [ ]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat_simple import ForexTradingEnv
# from trading.environments.forex_env_flat_multi_pair import MultipairForexTradingEnv

from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager
from sb3_contrib import RecurrentPPO

source_path = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/df_with_all_indics_unbiased/not_norm/train/'
source_dfs = [os.path.join(source_path, f) for f in os.listdir(source_path) if f.endswith('.parquet') and not f.startswith('.') and 'validate' not in f]

eval_path = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/df_with_all_indics_unbiased/not_norm/train/EUR_GBP_validate.parquet'
sequence = 5
saving_path = f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/df_with_all_indics_unbiased/not_norm/train/results/'
os.makedirs(saving_path, exist_ok=True)

INCLUDED_FEATURES=[
            'rsi', 'roc_10', 'stoch_rsi', 'stoch_k', 'stoch_d', 
            'bb_upper', 'bb_middle', 'bb_lower', 'bb_bandwidth', 'bb_percent',
            'atr', 'plus_di', 'minus_di', 'adx', 'price_change', 'close_norm',
            'hl_range_ratio', 'dist_sma_20_atr_adj', 'dist_sma_50_atr_adj',
            'sma_cross_atr', 'bb_position', 'bb_width_normalized',
            'tenkan_dist_atr_adj', 'kijun_dist_atr_adj', 'rsi_change',
            'macd_hist_norm', 'di_spread', 'trend_strength', 'atr_normalized',
            'atr_regime_10', 'atr_regime_20', 'trend_strength_20',
            'trend_strength_50', 'momentum_1h', 'momentum_4h', 'price_norm'
        ],
EXCLUDED_FEATURES=['timestamp', 'volume', 'open', 'high', 'low', 'close']


def make_train_env():
    env = ForexTradingEnv(
        df_paths=source_dfs,
        eval_mode=False,
        sequence_length=sequence,
        # included_features=INCLUDED_FEATURES,
        # excluded_features=EXCLUDED_FEATURES,

    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True)
    return env

def make_eval_env():
    env = ForexTradingEnv(
        df_paths=source_dfs,
        eval_path=eval_path,
        eval_mode=True,
        pair='EUR_GBP',
        sequence_length=sequence,
        # included_features=INCLUDED_FEATURES,
        # excluded_features=EXCLUDED_FEATURES,

    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=False)
    env.training = False
    return env

train_env = make_train_env()
eval_env = make_eval_env()

eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=saving_path,
    log_path=saving_path,
    eval_freq=100_000,  # Adjust as needed
    n_eval_episodes=5,
    deterministic=True,
    render=False
)

model = PPO(
    'MlpPolicy',
    train_env,
    verbose=0,
    tensorboard_log=f'{saving_path}sequence_{sequence}_cont_space_PPO_13m/',
)

model.learn(
    total_timesteps=13_000_000,  # Adjust as needed
    callback=eval_callback
)

model.save(f'{saving_path}{sequence}_best_model.zip')
train_env.save(f'{saving_path}{sequence}_vec_normalize.pkl')


Logging configuration loaded from /Users/floriankockler/Code/GitHub.nosync/ai6-gcp-bot/forex_trading_system/config/logging_config.yaml
2024-12-12 10:19:42 - ForexEnv2_flat - INFO - forex_env2_flat_simple.py:507 - Selected features (37): ['close', 'rsi', 'roc_10', 'stoch_rsi', 'stoch_k', 'stoch_d', 'bb_upper', 'bb_middle', 'bb_lower', 'bb_bandwidth', 'bb_percent', 'atr', 'plus_di', 'minus_di', 'adx', 'price_change', 'close_norm', 'hl_range_ratio', 'dist_sma_20_atr_adj', 'dist_sma_50_atr_adj', 'sma_cross_atr', 'bb_position', 'bb_width_normalized', 'tenkan_dist_atr_adj', 'kijun_dist_atr_adj', 'rsi_change', 'macd_hist_norm', 'di_spread', 'trend_strength', 'atr_normalized', 'atr_regime_10', 'atr_regime_20', 'trend_strength_20', 'trend_strength_50', 'momentum_1h', 'momentum_4h', 'price_norm']
2024-12-12 10:19:43 - ForexEnv2_flat - INFO - forex_env2_flat_simple.py:507 - Selected features (37): ['close', 'rsi', 'roc_10', 'stoch_rsi', 'stoch_k', 'stoch_d', 'bb_upper', 'bb_middle', 'bb_lower', '

In [ ]:
source_path = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/df_with_all_indics_unbiased/not_norm/train/'
source_dfs = [os.path.join(source_path, f) for f in os.listdir(source_path) if f.endswith('.parquet') and not f.startswith('.')]
source_dfs

In [ ]:
test = pd.read_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/df_with_all_indics_unbiased/not_norm/train/NZD_JPY_train.parquet')
test.columns